# Image Based Visual Servoing



### Import camera calibration data

In [23]:
import pickle
import cv2

# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)

mtx = camera_cal['mtx']
dist = camera_cal['dist']


### Load pre-trained SSD model

In [3]:
from jetbot import ObjectDetector

model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')

### Set up camera for object detection
* Camera is set up for 300x300 pixel video
* Use undistort() to undistort camera image prior to object detection

In [5]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

In [109]:
def undistort(img, mtx, dist, crop=False):
   
    h,  w = img.shape[:2]
    print (h,w)
    
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    
    return dst

In [117]:
undistort_image = undistort(camera.value, mtx, dist)
detections = model(undistort_image)

items = detections[0]
for item in items:
    print(item['label'],COCO_labels[item['label']], item['confidence'], item['bbox'])

300 300
62 couch 0.8373541235923767 [0.4387192726135254, 0.38629579544067383, 0.5674471855163574, 0.6021446585655212]
72 laptop 0.802555501461029 [0.1857829988002777, 0.3081863224506378, 0.3932817280292511, 0.44319745898246765]


In [100]:
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection



In [95]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([image_widget]))

width = int(image_widget.width)
height = int(image_widget.height)

In [118]:
target = 62

# draw all detections on image
for det in detections[0]:
    bbox = det['bbox']
    cv2.rectangle(undistort_image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 1)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == target]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(undistort_image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)

image_widget.value = bgr8_to_jpeg(undistort_image)
